# RAG 기반 챗봇 구현하기

## 필요 라이브러리 설치
- 가상 환경에 설치 권장
  - 가상 환경 활성화
  - 각 라이브러리 설치
  - 해당 가상 환경 커널 설정 후 실행

In [9]:
%pip install PyMuPDF

Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install langchain-community

Note: you may need to restart the kernel to use updated packages.


## PDF 파일 텍스트 변환 및 청크 쪼개기

### PDF 파일을 읽어서 텍스트 데이터 추출

In [13]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('C:/githurb/llm-programming/data/OneNYC_2050_Strategic_Plan.pdf')
data_nyc = loader.load()
data_nyc[0]

Document(metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 14.0 (Windows)', 'creationdate': '2019-04-30T13:48:30-04:00', 'moddate': '2020-01-03T15:55:12-05:00', 'title': '', 'trapped': '/False', 'source': 'C:/githurb/llm-programming/data/OneNYC_2050_Strategic_Plan.pdf', 'total_pages': 332, 'page': 0, 'page_label': '1'}, page_content='OneNYC \n2050\nBUILDING A STRONG \nAND FAIR CITY \nVOLUME 1 OF 9  \nA\nPRIL 2019\nTHE CITY OF NEW YORK\nMAYOR BILL DE BLASIO\nDEAN FULEIHAN  \nFIRST DEPUTY MAYOR \nD\nOMINIC WILLIAMS  \nCHIEF POLICY ADVISOR\nD\nANIEL A. ZARRILLI  \nOne NYC DIRECTOR')

### Splitter 설정 및 청크 쪼개기
- 오버랩 : 청크 경계에서 문맥이 끊기는 것을 방지

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 데이터를 1000자 단위로 나눕니다. overlap은 100자로 설정합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(data_nyc)

In [15]:
for i, split in enumerate(all_splits):
    print(f"Split {i+1}:------------------------------------\n")
    print(split)

Split 1:------------------------------------

page_content='OneNYC 
2050
BUILDING A STRONG 
AND FAIR CITY 
VOLUME 1 OF 9  
A
PRIL 2019
THE CITY OF NEW YORK
MAYOR BILL DE BLASIO
DEAN FULEIHAN  
FIRST DEPUTY MAYOR 
D
OMINIC WILLIAMS  
CHIEF POLICY ADVISOR
D
ANIEL A. ZARRILLI  
One NYC DIRECTOR' metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 14.0 (Windows)', 'creationdate': '2019-04-30T13:48:30-04:00', 'moddate': '2020-01-03T15:55:12-05:00', 'title': '', 'trapped': '/False', 'source': 'C:/githurb/llm-programming/data/OneNYC_2050_Strategic_Plan.pdf', 'total_pages': 332, 'page': 0, 'page_label': '1'}
Split 2:------------------------------------

page_content='2   |   O neNYC 2050 
NYC.GOV/O neNYC
ONENYC 2050 IS A STRATEGY TO SECURE OUR CITY’S 
FUTURE AGAINST THE CHALLENGES OF TODAY AND 
TOMORROW. WITH BOLD ACTIONS TO CONFRONT OUR 
CLIMATE CRISIS, ACHIEVE EQUITY, AND STRENGTHEN 
OUR DEMOCRACY, WE ARE BUILDING A STRONG AND 
FAIR CITY. JOIN US.
OneNYC 2050
BUILDING 

In [16]:
print(type(all_splits[0]))

<class 'langchain_core.documents.base.Document'>


### 다른 PDF 파일도 동일 작업 진행

In [17]:
loader_seoul = PyPDFLoader('C:/githurb/llm-programming/data/2040_seoul_plan.pdf')
data_seoul = loader_seoul.load()
seoul_splits = text_splitter.split_documents(data_seoul)
for i, split in enumerate(seoul_splits):
    print(f"Split {i+1}:------------------------------------")
    print(split)

Split 1:------------------------------------
page_content='「2040 서울도시기본계획」을 발간하며
지난 3년간 코로나19 팬데믹으로 전 세계가 심각한 타격을 받아왔지만, 대한민국의 수도 서울은 혁신적인 디지털 기술과 뛰어난 시민 의식, 풍부한 자연환경을 토대로 도시의 가능성과 잠재력을 확인할 수 있었습니다. 「2040 서울도시기본계획」은 기후위기, 디지털 전환, 생활양식의 변화 등 글로벌 대도시가 당면한 과제에 대한 해법을 제시하고 있습니다.첫째, 보행일상권으로의 공간구조 개편입니다. 서울을 하나의 기준으로 관리하던 지금까지의 방식에서 벗어나, 미래의 서울은 다양한 지역특성을 반영한 차별화된 계획체계를 통해 동네단위의 자족적 생활권으로 재구성하게 됩니다. 이는 기후위기, 팬데믹 등 각종 재난상황에서도 도시활동이 가능한 공간단위로, 서울 어디에서나 시민 삶의 질이 보장되는 새로운 차원의 균형발전정책 기반이 될 것입니다.둘째, 과감하고 유연한 도시계획 기조로의 전환입니다. 앞으로의 도시계획은 미래 여건변화에 유연하고 신속하게 대응할 수 있는 체계를 통해 현장에서 강력하게 작동하는 수단이 될 것입니다. 미래지향적인 계획철학을 토대로 융복합적인 토지이용제도를 실현하고 수변녹지와 연계된 생활공간을 조성하는 등 도시계획체계를 과감하고 유연하게 전환했습니다.' metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2020 11.0.0.5178', 'creationdate': '2024-12-12T18:16:11+09:00', 'author': 'SI', 'moddate': '2024-12-12T18:16:11+09:00', 'pdfversion': '1.4', 'source': 'C:/githurb/llm-programming/data/2040_seoul_plan.pdf', 'total_pages': 205, 'page': 1, 'page_label': '2'}
Spli

In [18]:
print(seoul_splits[50].page_content)
print('----------------------')
print(seoul_splits[51].page_content)

32제2장 미래상과 목표
9) 대기질 개선과 폐기물 관리 등 광역거버넌스 차원으로 풀어야 할 과제 발생증가하는 생활폐기물 배출량, 다시 우려되는 대기오염Ÿ서울에서 발생하는 폐기물을 처리해 왔던 인천 수도권매립지의 매립 종료가 2025년 예정되면서 폐기물 처리는 주요한 이슈로 떠올랐다. -2019년 폐기물 배출량은 2015년보다 0.6만 톤 증가한 4.8만 톤이 배출, 이 중 생활폐기물은 9,847톤으로 2015년에 비해 409톤 증가Ÿ이후 갑작스러운 팬데믹으로 인한 온라인 쇼핑, 배달 음식 소비증가 등을 고려하면 폐기물 배출량은 더욱 증가할 것으로 우려되고 있다.
[그림 2-14] 폐기물 발생량 및 1인당 생활폐기물 배출량(2010~2019)자료: 환경부, 서울시 폐기물 재활용 현황 통계, 각 연도Ÿ팬데믹 기간, 미세먼지와 관련한 대기질 지표는 일시적으로 좋아지는 양상을 보였다. -단기간 고농도 발생으로 생활에 직접적인 영향을 주는 주의보 발령 일수는 2019년 14일에서 2020년 4일로 감소하였다가 2021년 16일로 다시 증가Ÿ대기질 개선과 폐기물 감소를 위해서는 서울 대도시권 및 중앙정부 차원에서 대응방안을 마련하고 지속적으로 실행해 나가야 한다.10) 생활서비스 및 공원녹지 수요 증가생활서비스 시설은 확충 중, 노년인구를 위한 시설은 수요 증가에 비해 더딘 공급Ÿ생활서비스 시설의 공급 지표는 모든 분야에서 긍정적인 결과를 보인다. -보육시설, 아동·노인여가·재가노인복지시설, 장애인생활시설과 문화시설, 공공도서관, 공공체육시설, 병원, 공원 등의 시설도 지속적인 확충 중Ÿ2020년 65세 이상 노년인구는 전체 인구의 15.4%로 최근 10년 사이 비율이 6.0%p 늘어나면서 노인복지시설에 대한 수요가 증대되고 있다.
----------------------
제1절 서울의 변화진단33-노인여가복지시설 역시 확충되고 있으나, 노령화의 속도를 따라잡지 못해 2020년에는 노인 천 명당 2.1개소로 2010년에 비해 0.9개소 감소Ÿ다양한 생활서비스 시설의 양

### 후처리 과정
- 현재 청크($i$)의 끝에 다음 청크($i+1$)의 시작 100자를 덧붙임
- LSM이 다음 청크로 넘어가기 전의 문맥을 잃지 않고 답변을 생성

In [19]:
for i in range(len(seoul_splits) - 1):
    seoul_splits[i].page_content += "\n"+ seoul_splits[i + 1].page_content[:100]

print(seoul_splits[50].page_content)
print('----------------------')
print(seoul_splits[51].page_content)

32제2장 미래상과 목표
9) 대기질 개선과 폐기물 관리 등 광역거버넌스 차원으로 풀어야 할 과제 발생증가하는 생활폐기물 배출량, 다시 우려되는 대기오염Ÿ서울에서 발생하는 폐기물을 처리해 왔던 인천 수도권매립지의 매립 종료가 2025년 예정되면서 폐기물 처리는 주요한 이슈로 떠올랐다. -2019년 폐기물 배출량은 2015년보다 0.6만 톤 증가한 4.8만 톤이 배출, 이 중 생활폐기물은 9,847톤으로 2015년에 비해 409톤 증가Ÿ이후 갑작스러운 팬데믹으로 인한 온라인 쇼핑, 배달 음식 소비증가 등을 고려하면 폐기물 배출량은 더욱 증가할 것으로 우려되고 있다.
[그림 2-14] 폐기물 발생량 및 1인당 생활폐기물 배출량(2010~2019)자료: 환경부, 서울시 폐기물 재활용 현황 통계, 각 연도Ÿ팬데믹 기간, 미세먼지와 관련한 대기질 지표는 일시적으로 좋아지는 양상을 보였다. -단기간 고농도 발생으로 생활에 직접적인 영향을 주는 주의보 발령 일수는 2019년 14일에서 2020년 4일로 감소하였다가 2021년 16일로 다시 증가Ÿ대기질 개선과 폐기물 감소를 위해서는 서울 대도시권 및 중앙정부 차원에서 대응방안을 마련하고 지속적으로 실행해 나가야 한다.10) 생활서비스 및 공원녹지 수요 증가생활서비스 시설은 확충 중, 노년인구를 위한 시설은 수요 증가에 비해 더딘 공급Ÿ생활서비스 시설의 공급 지표는 모든 분야에서 긍정적인 결과를 보인다. -보육시설, 아동·노인여가·재가노인복지시설, 장애인생활시설과 문화시설, 공공도서관, 공공체육시설, 병원, 공원 등의 시설도 지속적인 확충 중Ÿ2020년 65세 이상 노년인구는 전체 인구의 15.4%로 최근 10년 사이 비율이 6.0%p 늘어나면서 노인복지시설에 대한 수요가 증대되고 있다.
제1절 서울의 변화진단33-노인여가복지시설 역시 확충되고 있으나, 노령화의 속도를 따라잡지 못해 2020년에는 노인 천 명당 2.1개소로 2010년에 비해 0.9개소 감소Ÿ다양한 
----------------------
제1절 서울의 변화

In [20]:
print(len(all_splits))
all_splits.extend(seoul_splits)
print(len(all_splits))

1023
1331


## 오픈 AI 임베딩 모델 사용하기

### 벡터DB 다운로드

In [21]:
%pip install langchain_chroma

Note: you may need to restart the kernel to use updated packages.


In [22]:
%pip install langchain_openai

Note: you may need to restart the kernel to use updated packages.


### 텍스트를 벡터로 변환

In [34]:
from langchain_openai import OpenAIEmbeddings 
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

embedding = OpenAIEmbeddings(model='text-embedding-3-large', api_key=OPENAI_API_KEY)
v = embedding.embed_query("뉴욕의 온실가스 저감 정책은 뭐야?")
print(v)
print(len(v))

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

### Chroma DB 생성 및 불러오기

In [24]:
from langchain_chroma import Chroma
import os

persist_directory = '../chroma_store'	

# 저장된 크로마 DB가 없다면 새로 만들기
if not os.path.exists(persist_directory):
    print("Creating new Chroma store")
    vectorstore = Chroma.from_documents(
        documents=all_splits,
        embedding=embedding,
        persist_directory=persist_directory
    )

else:
    print("Loading existing Chroma store")
    vectorstore = Chroma(		
        persist_directory=persist_directory, 
        embedding_function=embedding
    )

Creating new Chroma store


### 유사 청크 가져와서 질문하기

In [25]:
retriever = vectorstore.as_retriever(k=3)
docs = retriever.invoke("서울시의 환경 정책에 대해 궁금해")

for d in docs:
    print(d)
    print('------')

page_content='제4절 기후·환경 부문1. 개요Ÿ기후변화는 21세기에 전 지구적으로 가장 위중한 영향을 미칠 것으로 예상되며, 시민 생활의 모든 측면과 연관되어 있어 향후 서울시의 적극적인 대응이 필요하다.Ÿ탄소중립 목표뿐만 아니라 미세먼지로부터 시민 건강을 지키기 위해서는 건물, 교통, 에너지 등 도시의 주요 인프라 전반의 혁신이 요구되며, 이를 위해 새로운 기술과 혁신적 제도가 필요하다. 제로에너지 건물, 친환경 차량 및 교통 인프라의 확대, 자원·에너지 순환 기반 조성으로 온실가스와 미세먼지 배출량을 획기적으로 감축해야 한다. Ÿ기후변화에 따른 폭염, 풍수해, 도심열섬현상 등 기후재난 및 극한 기후현상이 심해질 것으로 전망되어 보다 능동적인 대비가 필요하다. Ÿ한편, 환경보존과 쾌적한 도시환경을 위해 도심 곳곳 시민 모두가 누릴 수 있는 도심숲과 생활공원 등 녹색공간을 조성하고, 이를 수변 공간과 연계하여 풍부하고 지속가능한 자연환경이 확보될 수 있도록 한다.Ÿ장기적인 측면에서 시민 개개인과 기업 등 다양한 도시 내 행위자의 적극적인 협조가 필수적이며 이를 위해 중앙정부와 서울시 환경계획 담당부서와의 협력적이고 포용적인 거버넌스 체계를 구축하도록 한다.목표 전략3-12050 탄소중립 실현을 위한 도시 인프라 전환3-1-1건물 부문의 탄소배출을 감축하기 위한 친환경 기술 개발 및 적극 적용3-1-2미래 모빌리티 기술 활용과 친환경 수송 차량 및 관련 인프라 확충3-1-3에너지 전환을 위한 청정에너지 기반 구축3-1-4대기 환경을 고려한 공간계획과 배출원 관리체계 강화3-2건강한 순환도시 조성을 위한자립적인 자원순환 체계 구축3-2-1자원순환·관리 자립을 위한 분산형 폐기물처리 시설 구축3-2-2기후 행동 포용적 거버넌스 구축을 위한 시민 행동 활성화3-3사람과 자연의 공존을 위한친환경 생태도시 구축3-3-1건물 에너지 분야 효율성 개선 및 도심 속 생물 다양성 확보3-3-2지속가능한 통합 물순환 체계 구축3-4다양한 수변을 경험할 수 있는수변감성도

## 청크 기반으로 답변 생성하기

### 의존성 문제 해결을 위해 라이브러리 추가 설치

In [26]:
%pip install langchain-classic

Note: you may need to restart the kernel to use updated packages.


### 청크를 바탕으로 답변하는 시스템 생성하기

In [27]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini")


question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "사용자의 질문에 대해 아래 context에 기반하여 답변하라.:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

## 메세지 저장하고 답변 생성하기

In [ ]:
from langchain_classic.memory import ChatMessageHistory

# 채팅 메시지 저장할 메모리 객체 생성
chat_history = ChatMessageHistory() 
# 사용자 질문을 메모리에 저장
chat_history.add_user_message("서울시의 온실가스 저감 정책에 대해 알려줘.") 

# 문서 검색하고 답변 생성
answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": docs,
    }
)

# 생성된 답변 메모리에 저장
chat_history.add_ai_message(answer) 

print(answer)

서울시의 온실가스 저감 정책은 기후변화 대응과 시민 건강을 위해 다양한 전략과 목표를 설정하고 있습니다. 주요 내용은 다음과 같습니다:

1. **탄소중립 목표**: 서울시는2030년까지 온실가스를 저감하고 탄소중립 목표를 달성하기 위해 적극적인 대응을 추진하고 있습니다.

2. **도시 인프라 혁신**: 
   - 건물 부문에서는 친환경 기술을 개발하고 적용하여 탄소배출을 감축합니다.
   - 미래 모빌리티 기술을 활용한 친환경 교통수단 및 인프라를 확충합니다.
   - 청정에너지 기반을 구축하고, 대기 환경을 고려한 공간계획 및 배출원 관리 체계를 강화합니다.

3. **자원순환 체계 구축**: 온실가스 배출을 줄이기 위해 자원순환 및 관리의 자립성을 높이고, 분산형 폐기물 처리 시설을 확충합니다. 이를 통해 자원재활용 비율을 높이고 폐기물 발생을 감소시키는 정책을 추진합니다.

4. **시민 참여와 행동 활성화**: 시민의 기후 행동을 장려하기 위해 다양한 교육 및 캠페인을 통해 기후 변화에 대한 인식 확대에 집중합니다. 친환경 교육을 통한 대중교통 이용 촉진 및 일회용품 사용 감소 등의 가이드라인을 제공합니다.

5. **공동체 협력**: 서울시는 중앙 정부 및 수도권 지자체와 협력하여 기후 환경 관리에 필요한 협력체계를 구축하고, 시민과 기업의 협업으로 효과적인 규제 및 관리 기반을 마련합니다.

6. **생물 다양성 및 녹지 공간 확대**: 건물의 에너지 효율성을 개선하고, 도심 속 생물 다양성을 확보하기 위해 녹지, 하천, 농지 및 자연 공간을 조성합니다.

이와 같은 정책들은 서울시가 기후변화에 능동적으로 대응하고 지속 가능한 도시 환경을 조성하기 위한 전반적인 노력을 반영하고 있습니다.


### 대화 기록 출력하기

In [ ]:
for m in chat_history.messages:
    print(m)

content='서울시의 온실가스 저감 정책에 대해 알려줘.' additional_kwargs={} response_metadata={}
content='서울시의 온실가스 저감 정책은 기후변화 대응과 시민 건강을 위해 다양한 전략과 목표를 설정하고 있습니다. 주요 내용은 다음과 같습니다:\n\n1. **탄소중립 목표**: 서울시는2030년까지 온실가스를 저감하고 탄소중립 목표를 달성하기 위해 적극적인 대응을 추진하고 있습니다.\n\n2. **도시 인프라 혁신**: \n   - 건물 부문에서는 친환경 기술을 개발하고 적용하여 탄소배출을 감축합니다.\n   - 미래 모빌리티 기술을 활용한 친환경 교통수단 및 인프라를 확충합니다.\n   - 청정에너지 기반을 구축하고, 대기 환경을 고려한 공간계획 및 배출원 관리 체계를 강화합니다.\n\n3. **자원순환 체계 구축**: 온실가스 배출을 줄이기 위해 자원순환 및 관리의 자립성을 높이고, 분산형 폐기물 처리 시설을 확충합니다. 이를 통해 자원재활용 비율을 높이고 폐기물 발생을 감소시키는 정책을 추진합니다.\n\n4. **시민 참여와 행동 활성화**: 시민의 기후 행동을 장려하기 위해 다양한 교육 및 캠페인을 통해 기후 변화에 대한 인식 확대에 집중합니다. 친환경 교육을 통한 대중교통 이용 촉진 및 일회용품 사용 감소 등의 가이드라인을 제공합니다.\n\n5. **공동체 협력**: 서울시는 중앙 정부 및 수도권 지자체와 협력하여 기후 환경 관리에 필요한 협력체계를 구축하고, 시민과 기업의 협업으로 효과적인 규제 및 관리 기반을 마련합니다.\n\n6. **생물 다양성 및 녹지 공간 확대**: 건물의 에너지 효율성을 개선하고, 도심 속 생물 다양성을 확보하기 위해 녹지, 하천, 농지 및 자연 공간을 조성합니다.\n\n이와 같은 정책들은 서울시가 기후변화에 능동적으로 대응하고 지속 가능한 도시 환경을 조성하기 위한 전반적인 노력을 반영하고 있습니다.' additional_kwargs={} response_metadata={}


## 질의 확장 구현하기

### 문자열 출력 파서 불러오기

In [ ]:
from langchain_core.output_parsers import StrOutputParser

### 구체적인 질문으로 프롬프트 증강

In [ ]:
query_for_nyc = "뉴욕은?"

# query augmentation 
# 기존 대화 내용을 활용해 query_augmentation 수행
query_augmentation_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"), # 기존 대화 내용
        (
            "system",
            "기존의 대화 내용을 활용하여 사용자의 아래 질문의 의도를 파악하여 명료한 한 문장의 질문으로 변환하라. 대명사나 이, 저, 그와 같은 표현을 명확한 명사로 표현하라. :\n\n{query}",
        ),
    ]
)

### 결과 문자열로 반환하는 체인 생성

In [ ]:
query_augmentation_chain = query_augmentation_prompt | chat | StrOutputParser()

In [ ]:
augmented_query = query_augmentation_chain.invoke({
    "messages": chat_history.messages,
    "query": query_for_nyc  
})

print(augmented_query)

뉴욕시에 대한 정보를 알려 주세요.


### 벡터DB 검색 후 결과 출력

In [ ]:
docs = retriever.invoke(augmented_query)

for d in docs:
    print(d)
    print('------')

page_content='United States, and our residents speak more than 200 languages. 
Neighborhoods in all five boroughs are adding residents, with 
new population centers emerging alongside a new economy 
in formerly commercial and industrial neighborhoods such as 
Lower Manhattan, Downtown Brooklyn, and the Williamsburg 
waterfront, as well as neighborhoods that only a few decades ago 
experienced severe population decline, including the South Bronx 
and East New York.
As population continues to reach all-time highs, new housing 
construction has reached its highest levels since the 1960s. More 
than 350,000 residential units have been built since 2000, most of 
which are along subway lines, reversing historic patterns of sprawl. 
New York City is adding more units in larger apartment buildings 
than ever before. This high-density growth is concentrated in 
certain neighborhoods, such as Long Island City, Hudson Yards, and 
Flushing.
To support this growth, the City has invested billions of

### 언어 모델에서 답변 생성하기

In [ ]:
chat_history.add_user_message(query_for_nyc) # query_for_nyc에 "뉴욕은?" 추가

answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": docs,
    }
)

# 생성된 답변 메모리에 저장
chat_history.add_ai_message(answer) 

print(answer)

뉴욕시의 온실가스 저감 정책은 도시의 지속 가능한 발전과 기후 변화 대응을 목표로 다양한 노력을 기울이고 있습니다. 주요 내용은 다음과 같습니다:

1. **탄소중립 목표**: 뉴욕시는 2050년까지 탄소중립 목표를 설정하고 이를 달성하기 위해 온실가스 배출량을 80% 이상 줄이겠다는 계획을 세웠습니다.

2. **건물 에너지 효율성 향상**: 건물은 뉴욕시의 총 온실가스 배출의 큰 비중을 차지하고 있어, 기존 건물의 에너지 효율성을 높이기 위한 규제와 프로그램이 실시되고 있습니다. 예를 들어, '가장 에너지 효율적인 건물법'(Local Law 97)과 같은 법이 시행되어 새로운 건물과 기존 건물의 에너지 사용량을 감소시키기 위한 목표를 설정합니다.

3. **청정 에너지 전환**: 재생 가능 에너지의 비율을 높이기 위해, 태양광 및 풍력 발전시설의 설치를 촉진하고 청정 에너지 구매 프로그램을 운영하고 있습니다. 뉴욕시는 100% 재생 가능 전력 사용을 목표로 하고 있습니다.

4. **교통 시스템 개선**: 대중교통 및 비즈니스의 친환경 이동을 장려하기 위해 대중교통 인프라를 확장하고 자전거 도로를 추가하는 등 교통 시스템의 지속 가능성을 높이고 있습니다. 또한, 전기차 충전소를 늘리고 전기차 사용을 장려하는 정책도 시행하고 있습니다.

5. **녹지 공간 및 환경 보호**: 도시의 녹지 공간을 확충하고 생물 다양성을 보존하기 위한 다양한 프로그램을 운영하고 있습니다. 공원과 도심의 녹지 공간은 기후 변화의 영향을 완화하는 데 중요한 역할을 합니다.

6. **시민 참여 및 교육**: 뉴욕시는 시민들에게 기후 변화에 대한 인식을 높이고, 저탄소 생활 방식을 촉진하기 위한 교육 및 캠페인을 진행하고 있습니다. 이를 통해 시민들이 온실가스 감축에 직접 참여할 수 있는 환경을 조성하고 있습니다.

이러한 정책들은 뉴욕시가 기후변화에 효과적으로 대응하고 지속 가능한 도시로 발전하기 위해 종합적으로 추진하는 전략을 반영하고 있습니다.
